# Predicting sale price of flat

## Background
In Singapore, close to 80% of the population live in public housing. These are heavily subsidised by the government and as such, strict rules have been implemented to ensure the subsidies are not being misused. One such rule is that every couple is not to own more than a single subsidised property at any one time. 

## Project Motivation
I am interested in finding the potential selling price of my flat in 3 years. In this project, I will use a variety of regression models to make predictions on the potential selling price, rather than to take the prediction of a single model. This is to showcase the relative effectiveness of the various models for the present purpose -- predict sale price of a flat given historical transaction data.

## Data Collection
Transaction data of public flats have been dilligently chronicled [here](https://data.gov.sg/dataset/resale-flat-prices). In the final version of the project, the data will be obtained from a json object from an API call. In the initial version, the entire dataset will be downloaded and fed to the project in a **pandas** dataframe. Operations such as cleaning and grouping will then be performed on the dataframe itself.

### Inspiration from GovTech DSAID blog

A huge part of this project is inspired from a [blog](https://medium.com/dsaid-govtech/wanted-data-scientists-how-we-designed-our-first-case-interview-9fd4eaa4607c) written by the [Data Science and Artificial Intelligence Division (DSAID)](https://medium.com/dsaid-govtech). After seeing the impact they have on the life on Singaporeans, after launching useful products like the TraceTogether Token and App, I thought what they are doing is very meaningful and creates impact in our everyday lives. I am sure the job there will be sufficiently motivating in and of itself.

I would want to start off with using the OneMap API to geocode the lat-long coordinates just for the fun of it. Intuitively, I do not think that should affect the selling price anymore than the 'Town' feature. Including it is purely for practice on calling information from APIs.

First, the essential libraries...

In [1]:
import pandas as pd
import numpy as np
import requests as req
from matplotlib import pyplot as plt
import json

Based on the information on [HDB API](https://data.gov.sg/dataset/resale-flat-prices), we can find the API query string as follows. We will perform our entire analysis using our entire dataset of 92,270 entries(correct as of the time of writing).


In [3]:

query_str = 'https://data.gov.sg/api/action/datastore_search?resource_id=42ff9cfe-abe5-4b54-beda-c88f9bb438ee&limit=500'
response = req.get(query_str)
response.status_code

200

A side note on the response obtained from the **requests** library. A *status_code* of 200 implies that the request was successful. Otherwise the response code would have been *404 NOT FOUND*. Other possible status codes can be found [here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

In [10]:
data = json.loads(response.content)
type(data)

dict

In [12]:
len(data['result']['records'])

500

## The Data
As mentioned, in the initial version, the data is obtained from a downloaded **.csv** file and read into a dataframe. This will be changed to an API call in the final version. After that, check that the dataset has been properly read and explore some features of the data such as the number of columns, the number of rows and the type of every column.

Even though data from the 1990's are available, they are probably outdated due to the numerous policy implementations over the years. We will only use the most recent data from 2017 onwards.

In [6]:
df = pd.read_csv("Data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [7]:
df.shape

(93818, 11)

We see that there are a total of 93818 entries, which should be enough for a small project.

In [15]:
df.describe()

,floor_area_sqm,lease_commence_date,resale_price
count,93818.000000,93818.000000,9.381800e+04
mean,97.770685,1994.500139,4.457392e+05
std,24.251371,13.096517,1.550889e+05
min,31.000000,1966.000000,1.400000e+05
25%,82.000000,1984.000000,3.350000e+05
50%,95.000000,1995.000000,4.150000e+05
75%,113.000000,2004.000000,5.220000e+05
max,249.000000,2019.000000,1.258000e+06


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93818 entries, 0 to 93817
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   month                93818 non-null  object 
 1   town                 93818 non-null  object 
 2   flat_type            93818 non-null  object 
 3   block                93818 non-null  object 
 4   street_name          93818 non-null  object 
 5   storey_range         93818 non-null  object 
 6   floor_area_sqm       93818 non-null  float64
 7   flat_model           93818 non-null  object 
 8   lease_commence_date  93818 non-null  int64  
 9   remaining_lease      93818 non-null  object 
 10  resale_price         93818 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 7.9+ MB
